In [ ]:
# Structure Visualiser
from pathlib import Path
from ase.visualize import view
from ase.io import read
from IPython.display import display

structure_dir = "Carbon_Structures/Crystalline/Relaxed/Final Trajectory Frame"

def view_relaxed_structures(dir, struct_num):

    dir = Path(dir)

    files = []
    for file in dir.rglob('*'):
        
        if file.suffix != ".cif":
            print(f"Unrecognized file suffix: {file.suffix}")
            continue
        files.append(file)
    chosen_file = files[struct_num]
    atoms = read(chosen_file)
    print(chosen_file.name)
    display(view(atoms, viewer='x3d'))

view_relaxed_structures(structure_dir,19)

In [63]:
# ------ GRAPHICAL ANALYSIS --------

# Graphical data points are means of all repeat runs with errors given as 1 standard deviation
import pandas as pd
import json
from pathlib import Path

# ------ FIGURE FORMATTING ------
import matplotlib.pyplot as plt
plt.style.use('crystalline.mplstyle')
from cycler import cycler
colour_cycle = cycler('color', [ '#000000', '#D55E00', '#3399FF', '#009E73']) 
marker_cycle = cycler('marker', ['o','s', 'x', '^'])
style_cycle = cycler('markersize', [6,3,3,3]) + cycler('markerfacecolor', ['none',  '#D55E00', '#3399FF', '#009E73'])
total_cycle = colour_cycle + marker_cycle + style_cycle
# -------------------------------

# Searches recursively through the specified directory
# Returns a dataframe from json files   
def import_data_files(directory):
    
    directory = Path(directory)

    imported_data_files_counter = 0

    rows=[]

    # Exclude step info
    exclude = {"steps_to_relax", "step_limit"}

    for path in directory.rglob("*"):
    
        if not path.is_file(): # Filters for files not directories
            continue
        
        if path.suffix != ".json":
            print(f"Unrecognized file formate: {path.name} .Must be .json")
            continue
        
        # Load data from each file
        with open(path, "r") as f:
            data = json.load(f)

        # Check that data files aren't empty
        if not data:
            print(f"No data found in {path}")
            continue
        
        # Skip isolated atom
        if data.get("structure") == "isolated_C.cif":
            continue

        # Remove relaxation step info
        numerical_data = {k: v for k, v in data.items() if k not in exclude}
        rows.append(numerical_data)

        imported_data_files_counter += 1
    
    df = pd.DataFrame(rows)
    
    missing_rows = df[df.isna().all(axis=1)]

    if not missing_rows.empty:
        print(f"WARNING: Empty cells in dataframe\n{missing_rows}")
        
    print(f"Imported {imported_data_files_counter} files")   
    return df

df = import_data_files("Analysis/Crystalline Analysis/Raw Data")

def scatter_plot(df, df_column_to_plot, y_label, chart_title, save_dir):

    # Save paths
    save_dir = Path(save_dir)
    png_dir = save_dir / "png Graphs"
    pdf_dir = save_dir / "pdf Graphs"
    png_dir.mkdir(parents=True, exist_ok=True)
    pdf_dir.mkdir(parents=True, exist_ok=True)
    png_path = png_dir/ f"{chart_title}.png"
    pdf_path = pdf_dir / f"{chart_title}.pdf"

    # Structures (enumerate for plotting)
    structures = df["structure"].unique().tolist()
    x_positions = {s: i for i, s in enumerate(structures)}

    # x labels
    x_label_map = {
        "ase_nanotube_9_0.cif"       : "Nanotube-(9,0)",
        "ase_nanotube_9_9.cif"       : "Nanotube-(9,9)",
        "C60_GAP_20.cif"             : "C60",
        "C100_GAP_20.cif"            : "C100",
        "Diamond_mp66.cif"           : "Diamond",
        "Hexagonal_Diamond_mp47.cif" : "Hexagonal Diamond",
        "Graphite_mp169.cif"         : "Graphite"
    }
    # Potentials
    potential_label_map = {
        "Carbon_GAP_20.xml"      : "GAP-20",
        "medium-0b3.pt"          : "MACE-0b3",
        "medium-mpa-0.pt"        : "MACE-mpa-0",
        "medium-omat-0.pt"        : "MACE-omat-0",
    }

    plt.figure()
    plt.rc('axes', prop_cycle=total_cycle)

    for potential in sorted(df["potential"].unique()):

        sub_df = df[df["potential"] == potential]

        x_values = [x_positions[s] for s in sub_df["structure"]]
        y_values = sub_df[df_column_to_plot]
        
        label = potential_label_map.get(potential, potential)
        plt.plot(x_values, y_values, linestyle='', label=label)
        

    plt.xticks(
        list(x_positions.values()),
        [x_label_map.get(s, s) for s in x_positions.keys()],
        rotation=90
        )
    
    plt.ylabel(y_label)
    plt.title(chart_title)
    plt.legend()
    plt.savefig(pdf_path)
    plt.savefig(png_path)
    plt.close('all')
    print(f"Created {chart_title} plots")

def all_analysis_wrapper():
    scatter_plot(df, "atomisation_energy/atom", "eV / atom", "Atomisation Energy", "Analysis/Crystalline Analysis")
    scatter_plot(df, "formation_energy/atom", "eV / atom", "Formation Energy", "Analysis/Crystalline Analysis")
    scatter_plot(df, "average_bond_length", "Bond Length (Å)", "Mean Bond Length", "Analysis/Crystalline Analysis")
    scatter_plot(df, "average_bond_angle", "Bond Angle (°)", "Mean Bond Angle", "Analysis/Crystalline Analysis")
all_analysis_wrapper()

Imported 28 files
Created Atomisation Energy plots
Created Formation Energy plots
Created Mean Bond Length plots
Created Mean Bond Angle plots
